## Loading Data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_set, valid_set = torch.utils.data.random_split(train_dataset, [40000, 10000])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=32, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=2)

## Training and Testing Utils

In [ ]:
import time
from tqdm import tqdm

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=10, device='cpu', model_name='model'):
    model.to(device)

    train_loss_history = []
    train_accuracy_history = []
    
    valid_loss_history = []
    valid_accuracy_history = []
    
    best_iter = 0
    best_valid_loss = float('inf')
    best_valid_accuracy = 0.0
    best_model = None

    start_time = time.time()

    for epoch in range(epochs):
        model.train()

        train_loss, train_corrects = 0.0, 0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            predicted = outputs.argmax(dim=1)
            train_corrects += (predicted == labels).sum().item()
    
        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = train_corrects / len(train_loader.dataset)

        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)
        
        model.eval()
        
        valid_loss, valid_corrects = 0.0, 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(valid_loader, desc=f'Epoch {epoch+1}/{epochs}'):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                predicted = outputs.argmax(dim=1)
                valid_corrects += (predicted == labels).sum().item()
        
        valid_loss = valid_loss / len(valid_loader.dataset)
        valid_accuracy = valid_corrects / len(valid_loader.dataset)
        
        if valid_accuracy > best_valid_accuracy:
            best_iter = epoch + 1
            best_valid_loss = valid_loss
            best_valid_accuracy = valid_accuracy
            best_model = model.state_dict()
        
        valid_loss_history.append(valid_loss)
        valid_accuracy_history.append(valid_accuracy)
        
        print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
        print(f'Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}')
    
    end_time = time.time()

    print(f'Total training time: {end_time - start_time:.2f} seconds.')
    
    torch.save(best_model, f'weights/{model_name}_best.pth')
    
    print(f'Best model at epoch {best_iter} with valid loss: {best_valid_loss:.4f}, valid accuracy: {best_valid_accuracy:.4f}')

    return train_loss_history, train_accuracy_history, valid_loss_history, valid_accuracy_history

In [ ]:
def test_model(model, test_loader, criterion, device='cpu'):
    model.to(device)
    model.eval()
    
    test_loss, test_corrects = 0.0, 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * inputs.size(0)
            
            predicted = outputs.argmax(dim=1)
            test_corrects += (predicted == labels).sum().item()
        
    test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = test_corrects / len(test_loader.dataset)
    
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}%')
    
    return test_loss, test_accuracy

## Loading Models

In [ ]:
from models import lnn, cnn
from matplotlib import pyplot as plt

#### Linear Neural Network

In [ ]:
lnn_model = lnn.LNN(32 * 32 * 3, num_classes=10)

num_params = sum([p.numel() for p in lnn_model.parameters()])
print("Number of parameters: ", num_params)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lnn_model.parameters(), lr=0.001)

train_loss_history, train_accuracy_history, valid_loss_history, valid_accuracy_history = train_model(lnn_model, train_loader, valid_loader, criterion, optimizer, epochs=50, device='cuda', model_name='lnn_model')

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, len(train_loss_history) + 1), train_loss_history, label='Training Loss', linewidth=3)
plt.plot(range(1, len(valid_loss_history) + 1), valid_loss_history, label='Validation Loss', linewidth=3)

plt.title('Training & Validation Loss History', fontsize=20)
plt.xlabel('Epoch', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.legend(loc='lower left', fontsize=15)

plt.savefig('figures/lnn_loss_history.pdf', format='pdf', bbox_inches='tight')

plt.show();

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, len(train_accuracy_history) + 1), train_accuracy_history, label='Training Accuracy', linewidth=3)
plt.plot(range(1, len(valid_accuracy_history) + 1), valid_accuracy_history, label='Validation Accuracy', linewidth=3)

plt.title('Training & Validation Accuracy History', fontsize=20)
plt.xlabel('Epoch', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.legend(loc='upper left', fontsize=15)

plt.savefig('figures/lnn_accuracy_history.pdf', format='pdf', bbox_inches='tight')

plt.show();

In [ ]:
test_loss, test_accuracy = test_model(lnn_model, test_loader, criterion, device='cuda')

#### Convolutional Neural Network

In [ ]:
cnn_model = cnn.CNN(3, num_classes=10)

num_params = sum([p.numel() for p in cnn_model.parameters()])
print("Number of parameters: ", num_params)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

train_loss_history, train_accuracy_history, valid_loss_history, valid_accuracy_history = train_model(cnn_model, train_loader, valid_loader, criterion, optimizer, epochs=30, device='cuda', model_name='cnn_model')

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, len(train_loss_history) + 1), train_loss_history, label='Training Loss', linewidth=3)
plt.plot(range(1, len(valid_loss_history) + 1), valid_loss_history, label='Validation Loss', linewidth=3)

plt.title('Training & Validation Loss History', fontsize=20)
plt.xlabel('Epoch', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.legend(loc='lower left', fontsize=15)

plt.savefig('figures/cnn_loss_history.pdf', format='pdf', bbox_inches='tight')

plt.show();

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(range(1, len(train_accuracy_history) + 1), train_accuracy_history, label='Training Accuracy', linewidth=3)
plt.plot(range(1, len(valid_accuracy_history) + 1), valid_accuracy_history, label='Validation Accuracy', linewidth=3)

plt.title('Training & Validation Accuracy History', fontsize=20)
plt.xlabel('Epoch', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.legend(loc='upper left', fontsize=15)

plt.savefig('figures/cnn_accuracy_history.pdf', format='pdf', bbox_inches='tight')

plt.show();

In [ ]:
test_loss, test_accuracy = test_model(cnn_model, test_loader, criterion, device='cuda')